In [ ]:
from fastai.vision.all import *
import numpy as np
import pandas as pd

In [ ]:
train_df = pd.read_csv("../input/mnist-analytics-vidhya/Train_UQcUa52/train.csv")
test_df = pd.read_csv("../input/mnist-analytics-vidhya/Test_fCbTej3_0j1gHmj.csv")

In [ ]:
train_df.head()

In [ ]:
train_df.label.value_counts()

In [ ]:

dls = ImageDataLoaders.from_df(train_df,path="../input/mnist-analytics-vidhya/Train_UQcUa52/Images",
                               folder="train",
                               label_col="label",valid_pct=0.2,
                               item_tfms= Resize(28),
#                                batch_tfms=aug_transforms(),
                              bs=32)

In [ ]:
dls.vocab

In [ ]:
dls.show_batch()

In [ ]:
learn = cnn_learner(dls,resnet50,metrics=accuracy)
learn.fine_tune(10)

In [ ]:
learn = cnn_learner(dls,resnet50,metrics=accuracy,model_dir="/kaggle/working")
learn.lr_find()

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=accuracy,model_dir="/kaggle/working")
learn.fine_tune(10, base_lr=1e-2)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(5, nrows=1, figsize = (25,10))

In [ ]:
test_path = "../input/mnist-analytics-vidhya/Train_UQcUa52/Images/test"
test_image_ids = [i for i in test_df.filename]
test_images = [test_path+f"/{img}" for img in test_image_ids]
preds = learn.get_preds(dl=dls.test_dl(test_images, shuffle=False, drop_last=False))

In [ ]:
preds = preds[0].cpu().numpy()

In [ ]:
pred = np.argmax(preds,axis=1)

In [ ]:
submission = pd.DataFrame()
submission["filename"] = test_image_ids
submission['label'] = pred
submission.to_csv("submission.csv", index=False)